# 使ってみる

In [6]:
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training, datasets, iterators, optimizers
from chainer.training import extensions
import numpy as np
import os
import math
import sys
from PIL import Image
from PIL import ImageDraw


/Users/choumori/project/chainer/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/choumori/project/chainer/lib/python3.6/site-packages/chainer/_environment_check.py:38: UserWarning: Accelerate has been detected as a NumPy backend library.
vecLib, which is a part of Accelerate, is known not to work correctly with Chainer.
We recommend using other BLAS libraries such as OpenBLAS.
For details of the issue, please see
https://docs.chainer.org/en/stable/tips.html#mnist-example-does-not-converge-in-cpu-mode-on-mac-os-x.

Also note that Chainer does not officially support Mac OS X.
Please use it at your own risk.

  ''')  # NOQA


In [8]:
uses_device = -1               # GPU#0を使用

# GPU使用時とCPU使用時でデータ形式が変わる
if uses_device >= 0:
    import cupy as cp
    import chainer.cuda
else:
    cp = np

In [9]:
class SuperResolution_NN(chainer.Chain):
    
    def __init__(self):
        # 重みデータの初期値を指定する
        w1 = chainer.initializers.Normal(scale=0.0378, dtype=None)
        w2 = chainer.initializers.Normal(scale=0.3536, dtype=None)
        w3 = chainer.initializers.Normal(scale=0.1179, dtype=None)
        w4 = chainer.initializers.Normal(scale=0.189, dtype=None)
        w5 = chainer.initializers.Normal(scale=0.0001, dtype=None)
        
        super(SuperResolution_NN, self).__init__()
        # 全ての層を定義する
        with self.init_scope():
            self.c1 = L.Convolution2D(1, 56, ksize=5, stride=1, pad=0, initialW=w1)
            self.l1 = L.PReLU()
            self.c2 = L.Convolution2D(56, 12, ksize=1, stride=1, pad=0, initialW=w2)
            self.l2 = L.PReLU()
            self.c3 = L.Convolution2D(12, 12, ksize=3, stride=1, pad=1, initialW=w3)
            self.l3 = L.PReLU()
            self.c4 = L.Convolution2D(12, 12, ksize=3, stride=1, pad=1, initialW=w3)
            self.l4 = L.PReLU()
            self.c5 = L.Convolution2D(12, 12, ksize=3, stride=1, pad=1, initialW=w3)
            self.l5 = L.PReLU()
            self.c6 = L.Convolution2D(12, 12, ksize=3, stride=1, pad=1, initialW=w3)
            self.l6 = L.PReLU()
            self.c7 = L.Convolution2D(12, 56, ksize=1, stride=1, pad=1, initialW=w4)
            self.l7 = L.PReLU()
            self.c8 = L.Deconvolution2D(56, 1, ksize=9, stride=3, pad=4, initialW=w5)
            
    def __call__(self, x, t=None, train=True):
        h1 = self.l1(self.c1(x))
        h2 = self.l2(self.c2(h1))
        h3 = self.l3(self.c3(h2))
        h4 = self.l4(self.c4(h3))
        h5 = self.l5(self.c5(h4))
        h6 = self.l6(self.c6(h5))
        h7 = self.l7(self.c7(h6))
        h8 = self.c8(h7)
        # 損失か結果を返す
        return F.mean_squared_error(h8, t) if train else h8


In [11]:
# ニューラルネットワークを作成
model = SuperResolution_NN()

# 学習結果を読み込む
chainer.serializers.load_hdf5( 'chapt03org.hdf5', model )

In [12]:
if uses_device >= 0:
    # GPUを使う
    chainer.cuda.get_device_from_id(0).use()
    chainer.cuda.check_cuda_available()
    # GPU用データ形式に変換
    model.to_gpu()

In [188]:
# 入力ファイル
in_file = 'test/1_7.png'

# 出力ファイル
dest_file = 'test/1_8.png'

In [189]:
img=Image.open(in_file)
img = img.resize((470, 470), Image.LANCZOS)

In [190]:
img.save(in_file)

In [187]:
# 入力画像を開く
img = Image.open(in_file).convert('YCbCr')

# 画像サイズが16ピクセルの倍数でない場合、16ピクセルの倍数にする
org_w = w = img.size[0]
org_h = h = img.size[1]
if w % 16 != 0:
    w = (math.floor(w / 16) + 1) * 16
if h % 16 != 0:
    h = (math.floor(h / 16) + 1) * 16
if w != img.size[0] or h != img.size[1]:
    img = img.resize((w,h))


# 出力画像
dst = Image.new('YCbCr', (10*w//4, 10*h//4), 'white')


# 入力画像を分割
cur_x = 0
while cur_x <= img.size[0] - 16:
    cur_y = 0
    while cur_y <= img.size[1] - 16:
        # 画像から切りだし
        rect = (cur_x, cur_y, cur_x+16, cur_y+16)
        cropimg = img.crop(rect)
        # YCbCrのY画素のみを使う
        hpix = cp.array(cropimg, dtype=cp.float32)
        hpix = hpix[:,:,0] / 255
        x = cp.array([[hpix]], dtype=cp.float32)
        # 超解像を実行
        t = model(x, train=False)
        # YCbCrのCbCrはBICUBICで拡大
        dstimg = cropimg.resize((40, 40), Image.BICUBIC)
        hpix = np.array(dstimg, dtype=np.float32)
        # YCbCrのY画素をコピー
        hpix.flags.writeable = True
        if uses_device >= 0:
            hpix[:,:,0] = chainer.cuda.to_cpu(t.data[0]) * 255
        else:
            hpix[:,:,0] = t.data[0] * 255
        # 画像を結果に配置
        bytes = np.array(hpix.clip(0,255), dtype=np.uint8)
        himg = Image.fromarray(bytes, 'YCbCr')
        dst.paste(himg, (10*cur_x//4, 10*cur_y//4, 10*cur_x//4 + 40, 10*cur_y//4 + 40))
        # 次の切りだし場所へ
        cur_y += 16
    cur_x += 16

# 結果を保存する
dst = dst.convert('RGB')
dst.save(dest_file)



# 訓練してみる

In [191]:
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training, datasets, iterators, optimizers
from chainer.training import extensions
import numpy as np
import os
import math
from PIL import Image


In [192]:
batch_size = 128                # バッチサイズ128
uses_device = -1                # GPU#0を使用

# GPU使用時とCPU使用時でデータ形式が変わる
if uses_device >= 0:
    import cupy as cp
    import chainer.cuda
else:
    cp = np

In [193]:
class SuperResolution_NN(chainer.Chain):
    def __init__(self):
        # 重みデータの初期値を指定する
        w1 = chainer.initializers.Normal(scale=0.0378, dtype=None)
        w2 = chainer.initializers.Normal(scale=0.3536, dtype=None)
        w3 = chainer.initializers.Normal(scale=0.1179, dtype=None)
        w4 = chainer.initializers.Normal(scale=0.189, dtype=None)
        w5 = chainer.initializers.Normal(scale=0.0001, dtype=None)
        super(SuperResolution_NN, self).__init__()
        # 全ての層を定義する
        with self.init_scope():
            self.c1 = L.Convolution2D(1, 56, ksize=5, stride=1, pad=0, initialW=w1)
            self.l1 = L.PReLU()
            self.c2 = L.Convolution2D(56, 12, ksize=1, stride=1, pad=0, initialW=w2)
            self.l2 = L.PReLU()
            self.c3 = L.Convolution2D(12, 12, ksize=3, stride=1, pad=1, initialW=w3)
            self.l3 = L.PReLU()
            self.c4 = L.Convolution2D(12, 12, ksize=3, stride=1, pad=1, initialW=w3)
            self.l4 = L.PReLU()
            self.c5 = L.Convolution2D(12, 12, ksize=3, stride=1, pad=1, initialW=w3)
            self.l5 = L.PReLU()
            self.c6 = L.Convolution2D(12, 12, ksize=3, stride=1, pad=1, initialW=w3)
            self.l6 = L.PReLU()
            self.c7 = L.Convolution2D(12, 56, ksize=1, stride=1, pad=1, initialW=w4)
            self.l7 = L.PReLU()
            self.c8 = L.Deconvolution2D(56, 1, ksize=9, stride=3, pad=4, initialW=w5)
            
    def __call__(self, x, t=None, train=True):
        h1 = self.l1(self.c1(x))
        h2 = self.l2(self.c2(h1))
        h3 = self.l3(self.c3(h2))
        h4 = self.l4(self.c4(h3))
        h5 = self.l5(self.c5(h4))
        h6 = self.l6(self.c6(h5))
        h7 = self.l7(self.c7(h6))
        h8 = self.c8(h7)
        # 損失か結果を返す
        return F.mean_squared_error(h8, t) if train else h8


In [194]:
# カスタムUpdaterのクラス
class SRUpdater(training.StandardUpdater):
    def __init__(self, train_iter, optimizer, device):
        super(SRUpdater, self).__init__(
            train_iter,
            optimizer,
            device=device
        )
        
    def update_core(self):
        # データを1バッチ分取得
        batch = self.get_iterator('main').next()
        # Optimizerを取得
        optimizer = self.get_optimizer('main')
        
        # バッチ分のデータを作る
        x_batch = [] # 入力データ
        y_batch = [] # 正解データ
        for img in batch:
            # 高解像度データ
            hpix = np.array(img, dtype=np.float32)  / 255.0
            y_batch.append([hpix[:,:,0]]) # Yのみの1chデータ
            # 低解像度データを作る
            low = img.resize((16, 16), Image.NEAREST)
            lpix = np.array(low, dtype=np.float32) / 255.0
            x_batch.append([lpix[:,:,0]]) # Yのみの1chデータ
            
        # numpy or cupy配列にする
        x = cp.array(x_batch, dtype=cp.float32)
        y = cp.array(y_batch, dtype=cp.float32)
        
        # ニューラルネットワークを学習させる
        optimizer.update(optimizer.target, x, y)

In [195]:
# ニューラルネットワークを作成
model = SuperResolution_NN()

if uses_device >= 0:
    # GPUを使う
    chainer.cuda.get_device_from_id(0).use()
    chainer.cuda.check_cuda_available()
    # GPU用データ形式に変換
    model.to_gpu()

images = []

In [196]:
# 全てのファイル
fs = os.listdir('train')
for fn in fs:
    # 画像を読み込み
    img = Image.open('train/' + fn).resize((320, 320)).convert('YCbCr')
    cur_x = 0
    while cur_x <= 320 - 40:
        cur_y = 0
        while cur_y <= 320 - 40:
            # 画像から切りだし
            rect = (cur_x, cur_y, cur_x+40, cur_y+40)
            cropimg = img.crop(rect).copy()
            # 配列に追加
            images.append(cropimg)
            # 次の切りだし場所へ
            cur_y += 20
        cur_x += 20

# 繰り返し条件を作成する
train_iter = iterators.SerialIterator(images, batch_size, shuffle=True)


In [197]:
# 誤差逆伝播法アルゴリズムを選択する
optimizer = optimizers.Adam()
optimizer.setup(model)

# デバイスを選択してTrainerを作成する
updater = SRUpdater(train_iter, optimizer, device=uses_device)
trainer = training.Trainer(updater, (10000, 'epoch'), out="result")
# 学習の進展を表示するようにする
trainer.extend(extensions.ProgressBar())

# 中間結果を保存する
n_save = 0
@chainer.training.make_extension(trigger=(1000, 'epoch'))
def save_model(trainer):
    # NNのデータを保存
    global n_save
    n_save = n_save+1
    chainer.serializers.save_hdf5( 'chapt03-'+str(n_save)+'.hdf5', model )
trainer.extend(save_model)

In [198]:
# 機械学習を実行する
trainer.run()

     total [..................................................]  0.00%
this epoch [###########.......................................] 22.14%
       100 iter, 0 epoch / 10000 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [..................................................]  0.00%
this epoch [######################............................] 44.27%
       200 iter, 0 epoch / 10000 epochs
     3.664 iters/sec. Estimated time to finish: 14 days, 6:28:32.652478.
     total [..................................................]  0.01%
this epoch [#################################.................] 66.41%
       300 iter, 0 epoch / 10000 epochs
    3.7147 iters/sec. Estimated time to finish: 14 days, 1:47:30.504639.
     total [..................................................]  0.01%
this epoch [############################################......] 88.54%
       400 iter, 0 epoch / 10000 epochs
      3.88 iters/sec. Estimated time to finish: 13 days, 11:23:55.537

KeyboardInterrupt: 

In [ ]:
# 学習結果を保存する
chainer.serializers.save_hdf5( 'chapt03.hdf5', model )